## Import thư viện

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import pickle

2023-06-29 23:43:13.125398: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 23:43:13.196264: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 23:43:13.197146: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 23:43:14.555381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Đọc dữ liệu

In [2]:
df = pd.read_csv('data/data_processed.csv')
df.head()

,user_id,isLike,uni_id,academic_encode,personality_encode,major_id_encode,block_encode
0,1,1,55,3,4,31,0
1,1,1,32,3,4,105,1
2,2,1,32,1,24,107,0
3,3,1,32,3,27,97,0
4,3,1,32,3,27,101,0


In [21]:
df = df.drop_duplicates(subset=["user_id"], keep='first', inplace=False)

In [24]:
df

,user_id,isLike,uni_id,academic_encode,personality_encode,major_id_encode,block_encode
0,1,1,55,3,4,31,0
2,2,1,32,1,24,107,0
3,3,1,32,3,27,97,0
5,5,4,75,3,7,0,0
6,6,3,32,1,21,97,1
...,...,...,...,...,...,...,...
339,155,2,55,0,19,25,1
340,156,4,66,0,4,109,1
342,157,3,65,0,27,9,11
344,158,3,21,0,7,47,0


In [32]:
df.iloc[1]

user_id                 2
isLike                  1
uni_id                 32
academic_encode         1
personality_encode     24
major_id_encode       107
block_encode            0
Name: 2, dtype: int64

## KNN for user based problem

In [22]:
X = df[['academic_encode', 'personality_encode']]
y = df["major_id_encode"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, y_train = X, y

In [23]:
X

,academic_encode,personality_encode
0,3,4
2,1,24
3,3,27
5,3,7
6,1,21
...,...,...
339,0,19
340,0,4
342,0,27
344,0,7


In [25]:
y

0       31
2      107
3       97
5        0
6       97
      ... 
339     25
340    109
342      9
344     47
345     26
Name: major_id_encode, Length: 156, dtype: int64

In [27]:
# Khởi tạo mô hình KNN
knn = KNeighborsClassifier()

# Thiết lập các giá trị để fine-tune
param_grid = {
    'n_neighbors': [5, 7, 9, 11],  # Số lượng láng giềng gần nhất
    'weights': ['uniform', 'distance'],  # Cách tính trọng số cho láng giềng
    'metric': ['euclidean', 'manhattan']  # Phương pháp tính khoảng cách
}
# Khởi tạo GridSearchCV với mô hình KNN và danh sách các giá trị để tìm kiếm
grid_search = GridSearchCV(knn, param_grid, cv=5)

# Huấn luyện mô hình trên tập huấn luyện
grid_search.fit(X_train, y_train)

# In ra các siêu tham số tốt nhất tìm được
print("Best Parameters:", grid_search.best_params_)

/home/tran/miniconda3/envs/RC/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best Parameters: {'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'uniform'}


In [28]:
best_knn = KNeighborsClassifier(metric='euclidean', n_neighbors = 11, weights='distance')
best_knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=11, weights='distance')

In [29]:
# Lưu mô hình KNN vào file pkl
def save_model(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

# Load mô hình KNN từ file pkl
def load_model(filename):
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    return model

In [30]:
# Lưu mô hình vào file pkl
save_model(best_knn, 'model/knn.pkl')

In [31]:
# best_knn.kneighbors([X_test.values[0]])